# 태아 상태 분류 문제

심전도 데이터를 세 가지 태아 상태 (정상, 의심, 아픔) 중 하나로 분류하는 데 도움이 되도록 분류기를 만들고자 한다.

## Get Data

In [3]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/00193/CTG.xls --output CTG.xls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
 25 1703k   25  434k    0     0   191k      0  0:00:08  0:00:02  0:00:06  191k
100 1703k  100 1703k    0     0   664k      0  0:00:02  0:00:02 --:--:--  664k


In [6]:
import os
os.listdir()

['.ipynb_checkpoints',
 'A01-NB-3.ipynb',
 'A02_Evaluation_TF_IDF.ipynb',
 'CTG.xls',
 'enron1',
 'enron1.tar.gz',
 'evaluation_assign.py',
 'naivebayes.py',
 'NB.ipynb',
 'SVM bc.ipynb',
 'SVM Kernels.ipynb',
 'SVM mc.ipynb',
 'SVM – Fetal State Classification.ipynb',
 'SVM-Topic Classification.ipynb',
 '__pycache__']

In [7]:
import pandas as pd
df = pd.read_excel('./CTG.xls','Raw Data')

In [8]:
df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Variab10.txt,1996-12-01,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
2,Fmcs_1.txt,1996-05-03,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,1996-05-03,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,1996-05-03,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0


In [9]:
X = df.iloc[1:2126,3:-2].values
Y = df.iloc[1:2126,-1].values

In [11]:
from collections import Counter
Counter(Y)

Counter({2.0: 295, 1.0: 1654, 3.0: 176})

split data

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2, random_state=42)

## Train SVM

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import timeit

In [14]:
svc = SVC(kernel='rbf')
parameters = {'C':(100,1e3,1e4,1e5),
             'gamma':(1e-08,1e-7,1e-6,1e-5)}
grid_search = GridSearchCV(svc, parameters, n_jobs=-1, cv=3)

In [15]:
start_time = timeit.default_timer()
grid_search.fit(X_train,Y_train)
print('--- %0.3fs seconds ---'%(timeit.default_timer()-start_time))

--- 15.853s seconds ---


In [16]:
grid_search.best_params_

{'C': 100000.0, 'gamma': 1e-07}

In [17]:
grid_search.best_score_

0.9447061902892292

## GridSearch

In [18]:
svc_best = grid_search.best_estimator_

In [20]:
accuracy = svc_best.score(X_test,Y_test)
print("The accuracy on testing set: {0:.1f}%".format(accuracy*100))

The accuracy on testing set: 95.5%


클래스 별 성능 확인

In [21]:
from sklearn.metrics import classification_report
prediction = svc_best.predict(X_test)
report = classification_report(Y_test,prediction)
print(report)

              precision    recall  f1-score   support

         1.0       0.96      0.98      0.97       324
         2.0       0.89      0.91      0.90        65
         3.0       1.00      0.78      0.88        36

    accuracy                           0.96       425
   macro avg       0.95      0.89      0.92       425
weighted avg       0.96      0.96      0.95       425

